# 0. Librerías

Cargamos las librerias necesarias para ejecutar el cuaderno.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import plotly.colors as pc

# 1. Data Load: Exploración del alquiler de bicicletas

Exploramos el dataset de Bikeshare para entender cómo el clima, la estacionalidad y los días festivos afectan la demanda de bicicletas.

Este dataset viene de serie con cierto tratamiento de los datos (ej.: la normalización de las temperaturas, humedad, velocidad del viento...)

In [ ]:
# Carga del dataset por hora
df = pd.read_csv("../data/hour.csv")
# Leemos las primeras entradas del dataset
df.head()

### Descripción de las columnas del dataset

El archivo `/data/readme.txt` aporta información sobre las columnas presentes en el dataset.

- **instant**: índice del registro
- **dteday**: fecha
- **season**: estación (1: primavera, 2: verano, 3: otoño, 4: invierno)
- **yr**: año (0: 2011, 1: 2012)
- **mnth**: mes (1 a 12)
- **hr**: hora (0 a 23)
- **holiday**: si el día es festivo (1) o no (0)
- **weekday**: día de la semana (0: domingo, 6: sábado)
- **workingday**: si es día laboral (1) o no (0)
- **weathersit**: situación climática (1: Despejado, 2: Niebla/Nubes, 3: Lluvia ligera/Nieve, 4: Lluvia intensa/Nieve)
- **temp**: temperatura normalizada (dividida por el valor max 41)
- **atemp**: sensación térmica normalizada (dividida por el valor max 50)
- **hum**: humedad normalizada (dividida por 100, tanto por 1)
- **windspeed**: velocidad del viento normalizada (dividida por el valor max 67)
- **casual**: número de usuarios ocasionales
- **registered**: número de usuarios registrados
- **cnt**: total de alquileres de bicicletas (casual + registrados)


# 2. Data Wrangling: Revisión y transformación de columnas.

Puesto que el dataset contiene varias columnas con valores normalizados y nuestro objetivo en este notebook es realizar una exploración descriptiva (no preparar los datos para un modelo predictivo concreto), vamos a realizar los siguientes pasos para facilitar la interpretación de los resultados:

- Renombrar las columnas para que sean más comprensibles.
- Desnormalizar las columnas que contienen valores escalados, devolviéndolas a sus unidades originales.
- Crear un índice temporal a partir de la fecha y la hora.
- Eliminar columnas que no son necesarias para el análisis exploratorio.
- Eliminar entradas duplicadas a partir del ínidice temporal.
- Eliminar posibles outliers dentro de un rango de interquartiles.

Estas transformaciones permitirán interpretar los datos de manera más clara y realizar visualizaciones más informativas.

In [ ]:
df["holiday"].nunique()

In [ ]:
# Renombramos columnas para facilitar su comprensión
df = df.rename(columns={
    "instant": "instante",
    "dteday": "fecha",
    "season": "estacion",
    "yr": "año",
    "mnth": "mes",
    "hr": "hora",
    "holiday": "festivo",
    "weekday": "dia_semana",
    "workingday": "dia_laboral",
    "weathersit": "sit_meteo",
    "hum": "norm_hum",
    "temp": "norm_temp",
    "atemp": "norm_atemp",
    "windspeed": "norm_windspeed",
    "casual": "casual",
    "registered": "registrado",
    "cnt": "num_usuarios"
})

In [ ]:
# Desnormalizamos las columnas previamente normalizadas
df["temperatura"] = (df["norm_temp"] * 41).round(1) # en grados Celsius
df["temperatura_sensacion"] = (df["norm_atemp"] * 50).round(1) # en grados Celsius
df["humedad"] = (df["norm_hum"] * 100).round(2) # en porcentaje
df["velocidad_viento"] = (df["norm_windspeed"] * 67).round(2) # en Km/h

In [ ]:
# Convertimos la columna 'dteday' a tipo datetime
df["timestamp"] = pd.to_datetime(df["fecha"]) + pd.to_timedelta(df["hora"], unit="h")
df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M")
df = df.set_index("timestamp")
df.index = pd.to_datetime(df.index)

In [ ]:
# Eliminamos columnas innecesarias por ahora
df = df.drop(columns=["instante", "año", "mes", "hora", "fecha", "norm_temp", "norm_atemp", "norm_hum", "norm_windspeed"], axis=1)

In [ ]:
# Mostrar filas duplicadas
duplicados = df[df.index.duplicated()]
duplicados

# Eliminar duplicados del dataset
df = df[~df.index.duplicated()]

In [ ]:
# Eliminar outliers fuera del rango intercuartílico para la columna objetiva de nuestro dataset "num_usuarios"
Q1 = df["num_usuarios"].quantile(0.25)
Q3 = df["num_usuarios"].quantile(0.75)
IQR = Q3 - Q1

# Limites superior e inferior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Nos quedamos solo con los datos dentro del rango permitido para la columna objetivo
df = df[(df["num_usuarios"] >= limite_inferior) | (df["num_usuarios"] <= limite_superior)]

## Descripción del dataset

Una vez seleccionadas las columnas necesarias para el análisis, vamos a ver una descripción de los valores que presentan cada una.

In [ ]:
# Crear un DataFrame resumen con info y nunique
info = pd.DataFrame({
    "tipo": df.dtypes,
    "nulos": df.isnull().sum(),
    "n_unique": df.nunique()
})
info

El DataFrame anterior aporta la siguiente información:

- No hay valores nulos ni NaNs.
- El tipo de datos de cada columna (int64 o float64) es el adecuado para su contenido.
- Los valores únicos de las columnas `estacion`, `festivo`, `dia_semana`, `dia_laboral` y `sit_meteo` coinciden con la información proporcionada en el archivo `Readme.txt`. Estas variables parecen ser categóricas y, probablemente, se les aplicó un one-hot encoding previamente.

In [ ]:
# Resumen estadístico del DataFrame
df.describe().round(2)

Todos los valores estadisticos mostrados en la tabla contienen valores normales dentro de sus rango.

# 3. Data Visualization: Relaciones a partir de gráficos

## Distribución de la demanda de bicicletas


In [ ]:
# Visualización de la frecuencia de usuarios totales
import plotly.express as px
fig = px.histogram(df, x=df["num_usuarios"], nbins=20)
fig.update_layout(
    title="Frecuencia de usuarios totales",
    xaxis_title="Número de usuarios",
    yaxis_title="Frecuencia"
)
fig.show()

In [ ]:
# Visualización de la frecuencia de usuarios totales con KDE (Histograma y estimación de densidad)

hist = go.Histogram(x=df["num_usuarios"], nbinsx=20, name="Frecuencia", marker_color="lightblue", opacity=0.9)

kde = gaussian_kde(df["num_usuarios"])
x_vals = np.linspace(df["num_usuarios"].min(), df["num_usuarios"].max(), 200)
y_vals = kde(x_vals) * len(df["num_usuarios"]) * (x_vals[1] - x_vals[0])  # Escalado para que coincida con la frecuencia
kde_line = go.Scatter(x=x_vals, y=y_vals, mode="lines", name="KDE", line=dict(color="red", width=2))

fig = go.Figure([hist, kde_line])
fig.update_layout(
    title="Frecuencia y KDE de usuarios totales",
    xaxis_title="Número de usuarios",
    yaxis_title="Frecuencia"
)
fig.show()

## Relación entre variables climáticas y la demanda

In [ ]:
# Gráficos de cajas personalizados por color para visualizar la relación entre variables climáticas y demanda

# ----- FIGURA 1: TEMPERATURA vs USUARIOS -----
fig1 = go.Figure()

# Obtener valores únicos de temperatura ordenados para usar como categorías
temps_unicas = sorted(df['temperatura'].unique())

# Usar escala secuencial de Plotly pero sin los colores muy claros del inicio
red_full = pc.sequential.Reds  # Color para la temperatura más baja
red_scale = red_full[2:] # Omitimos los primeros 2 tonos más claros

# Crear una caja para cada temperatura
for i, temp in enumerate(temps_unicas):
    # Normalizar índice y obtener color correspondiente
    normalized_idx = i / (len(temps_unicas) - 1)
    temp_color = pc.sample_colorscale(red_scale, normalized_idx)[0]
    
    # Filtrar datos para esta temperatura
    temp_data = df[df['temperatura'] == temp]
    
    # Añadir caja al gráfico
    fig1.add_trace(go.Box(
        y=temp_data['num_usuarios'],
        name=str(temp),
        marker_color=temp_color,
        boxmean=True  # Mostrar media
    ))

fig1.update_layout(
    title='Temperatura vs Nº de usuarios',
    xaxis_title='Temperatura (°C)',
    yaxis_title='Número de usuarios',
    showlegend=False
)

# ----- FIGURA 2: HUMEDAD vs USUARIOS -----
fig2 = go.Figure()

# Obtener valores únicos de humedad ordenados
humedad_unicas = sorted(df['humedad'].unique())

# Usar escala secuencial de Plotly pero sin los colores muy claros del inicio
blue_full = pc.sequential.Blues  # Color para la humedad más baja
blue_scale = blue_full[2:] # Omitimos los primeros 10 tonos más claros

# Crear una caja para cada valor de humedad
for i, hum in enumerate(humedad_unicas):
    # Normalizar índice y obtener color correspondiente
    normalized_idx = i / (len(humedad_unicas) - 1)
    hum_color = pc.sample_colorscale(blue_scale, normalized_idx)[0]
    
    # Filtrar datos para esta humedad
    hum_data = df[df['humedad'] == hum]
    
    # Añadir caja al gráfico
    fig2.add_trace(go.Box(
        y=hum_data['num_usuarios'],
        name=str(hum),
        marker_color=hum_color,
        boxmean=True  # Mostrar media
    ))

fig2.update_layout(
    title='Humedad vs Nº de usuarios',
    xaxis_title='Humedad (%)',
    yaxis_title='Número de usuarios',
    showlegend=False
)

# ----- FIGURA 3: VELOCIDAD DEL VIENTO vs USUARIOS -----
fig3 = go.Figure()

# Obtener valores únicos de velocidad del viento ordenados
viento_unicos = sorted(df['velocidad_viento'].unique())

# Usar escala secuencial de Plotly pero sin los colores muy claros del inicio
green_full = pc.sequential.Greens # Omitimos los primeros 10 tonos más claros
green_scale = green_full[2:]  # Color para la velocidad del viento más baja

# Crear una caja para cada valor de velocidad del viento
for i, viento in enumerate(viento_unicos):
    # Normalizar índice y obtener color correspondiente
    normalized_idx = i / (len(viento_unicos) - 1)
    viento_color = pc.sample_colorscale(green_scale, normalized_idx)[0]
    
    # Filtrar datos para esta velocidad del viento
    viento_data = df[df['velocidad_viento'] == viento]
    
    # Añadir caja al gráfico
    fig3.add_trace(go.Box(
        y=viento_data['num_usuarios'],
        name=str(viento),
        marker_color=viento_color,
        boxmean=True  # Mostrar media
    ))

fig3.update_layout(
    title='Velocidad del viento vs Nº de usuarios',
    xaxis_title='Velocidad del viento (km/h)',
    yaxis_title='Número de usuarios',
    showlegend=False
)

# Mostrar los gráficos
fig1.show()
fig2.show()
fig3.show()

## Efecto de la hora, el día de la semana, estación y vacaciones

In [ ]:
# ----- FIGURA 4: USUARIOS POR DÍA DE LA SEMANA -----

# Crear colores distintos para cada día de la semana (colores más diferenciados entre sí)
colores_dias = {
    1: '#4CAF50',  # Lunes - verde
    2: '#FF5722',  # Martes - naranja rojizo
    3: '#9C27B0',  # Miércoles - púrpura
    4: '#2196F3',  # Jueves - azul
    5: '#FFEB3B',  # Viernes - amarillo
    6: '#F44336',  # Sábado - rojo
    0: '#795548',  # Domingo - marrón
}

# Mapear días de la semana a nombres
dia_nombres = {
    1: 'Lunes',
    2: 'Martes',
    3: 'Miércoles',
    4: 'Jueves',
    5: 'Viernes',
    6: 'Sábado',
    0: 'Domingo',
}

# Crear una copia del DataFrame con los días de la semana como texto
df_temp = df.copy()
df_temp['dia_nombre'] = df_temp['dia_semana'].map(dia_nombres)

# Orden personalizado para los días (Lunes a Domingo)
orden_dias = [1, 2, 3, 4, 5, 6, 0]

# Crear el gráfico
fig4 = go.Figure()

# Añadir una caja para cada día de la semana en el orden especificado
for dia_num in orden_dias:
    dia_data = df[df['dia_semana'] == dia_num]
    fig4.add_trace(go.Box(
        y=dia_data['num_usuarios'],
        name=dia_nombres[dia_num],
        marker_color=colores_dias[dia_num],
        boxmean=True  # Mostrar media
    ))

fig4.update_layout(
    title='Alquileres por día de la semana',
    xaxis_title='Día de la semana',
    yaxis_title='Número de usuarios',
    showlegend=False
)

# ----- FIGURA 5: USUARIOS POR ESTACIÓN -----

# Crear un diccionario de colores acordes a las estaciones
colores_estaciones = {
    1: '#4CAF50',  # Primavera - verde
    2: '#FF9800',  # Verano - naranja
    3: '#8D6E63',  # Otoño - marrón
    4: '#2196F3'   # Invierno - azul
}

# Mapear números de estación a nombres
estacion_nombres = {
    1: 'Primavera',
    2: 'Verano',
    3: 'Otoño',
    4: 'Invierno'
}

# Crear el gráfico
fig5 = go.Figure()

# Añadir una caja para cada estación
for estacion_num in sorted(df['estacion'].unique()):
    estacion_data = df[df['estacion'] == estacion_num]
    fig5.add_trace(go.Box(
        y=estacion_data['num_usuarios'],
        name=estacion_nombres[estacion_num],
        marker_color=colores_estaciones[estacion_num],
        boxmean=True  # Mostrar media
    ))

fig5.update_layout(
    title='Alquileres por estación',
    xaxis_title='Estación',
    yaxis_title='Número de usuarios',
    showlegend=False
)

# ----- FIGURA 6: USUARIOS EN DÍAS FESTIVOS vs NORMALES -----

# Colores para festivos y no festivos
colores_festivos = {
    0: '#607D8B',  # No festivo - gris azulado
    1: '#FFC107'   # Festivo - amarillo ámbar
}

# Mapear festivo a nombres
festivo_nombres = {
    0: 'No festivo',
    1: 'Festivo'
}

# Crear el gráfico
fig6 = go.Figure()

# Añadir una caja para cada tipo de día
for festivo_num in sorted(df['festivo'].unique()):
    festivo_data = df[df['festivo'] == festivo_num]
    fig6.add_trace(go.Box(
        y=festivo_data['num_usuarios'],
        name=festivo_nombres[festivo_num],
        marker_color=colores_festivos[festivo_num],
        boxmean=True  # Mostrar media
    ))

fig6.update_layout(
    title='Alquileres en días festivos vs normales',
    xaxis_title='Tipo de día',
    yaxis_title='Número de usuarios',
    showlegend=False
)

# Mostrar los gráficos
fig4.show()
fig5.show()
fig6.show()

## Matriz de correlación entre variables

In [ ]:

corrs = df[['temperatura', 'temperatura_sensacion', 'humedad', 'velocidad_viento', 'num_usuarios']].corr().round(2)
fig = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.values,
    colorscale='Viridis')
fig.update_layout(title='Matriz de correlación')
fig.show()

## EXTRA

1.  **Análisis de Series Temporales más Profundo**:
    *   **Descomposición de la serie temporal**: Analizar la columna `num_usuarios` a lo largo del tiempo para identificar y visualizar componentes como la tendencia (crecimiento a largo plazo), estacionalidad (patrones anuales, semanales, diarios) y residuos (variabilidad no explicada).
    *   **Autocorrelación**: Estudiar las funciones de autocorrelación (ACF) y autocorrelación parcial (PACF) de `num_usuarios` para entender cómo los valores pasados influyen en los valores futuros y detectar patrones cíclicos.
    *   **Patrones horarios específicos**: Aunque ya analizas el día de la semana, podrías investigar cómo varía la demanda promedio para cada hora del día, y cómo estos patrones horarios cambian entre días laborables y fines de semana, o entre diferentes estaciones.

2.  **Modelado Predictivo de la Demanda**:
    *   **Construcción de modelos**: Desarrollar modelos de regresión (ej. Regresión Lineal, Random Forest, Gradient Boosting, o modelos específicos para series temporales como ARIMA o Prophet) para predecir la cantidad de `num_usuarios`.
    *   **Importancia de características**: Identificar qué variables (temperatura, humedad, día de la semana, hora, etc.) son los predictores más importantes de la demanda de bicicletas.
    *   **Evaluación del rendimiento**: Medir la precisión de los modelos predictivos para entender cuán bien se puede prever la demanda futura.

3.  **Análisis Segmentado de Tipos de Usuario (Casual vs. Registrado)**:
    *   **Comportamiento diferencial**: Realizar análisis exploratorios y visualizaciones separadas para las columnas `casual` y `registrado`. Esto podría revelar si estos dos grupos de usuarios tienen diferentes sensibilidades al clima, al día de la semana, a los festivos, o diferentes patrones horarios.
    *   **Proporción de usuarios**: Analizar cómo cambia la proporción de usuarios casuales frente a registrados a lo largo del tiempo, o bajo diferentes condiciones (por ejemplo, si los fines de semana atraen a más usuarios casuales).
    *   **Modelos predictivos separados**: Considerar la creación de modelos predictivos distintos para usuarios casuales y registrados, ya que sus patrones de uso podrían ser lo suficientemente diferentes como para justificarlo.

4.  **Análisis de Interacción entre Variables**:
    *   **Efectos combinados**: Investigar cómo interactúan diferentes variables. Por ejemplo, ¿el efecto de la temperatura en la demanda es diferente en días laborables en comparación con los fines de semana? ¿O cómo la situación meteorológica (`sit_meteo`) modula el impacto de la hora del día? Esto se puede explorar con visualizaciones agrupadas o facetadas y modelos estadísticos que incluyan términos de interacción.

Estos análisis adicionales pueden proporcionar una comprensión más profunda de los factores que impulsan el uso de bicicletas y permitirían desarrollar estrategias más informadas para la gestión del servicio.